In [21]:
import pandas as pd
import json
import re

data_dir = "../data/for_biologist_correction/"
chapters = ['39', '40', '41', '42', '43']

In [14]:
stats = {'chapter': chapters, 'num_terms': [], 'num_sentences': []}

for chapter in chapters:
    with open(f"{data_dir}/life_bio_ch{chapter}/life_bio_ch{chapter}_sentences.txt", "r") as fid:
        sentences = fid.readlines()
        sentences = [s.strip() for s in sentences if len(s.strip())]
        stats['num_sentences'].append(len(sentences))
        
    with open(f"{data_dir}/life_bio_ch{chapter}/life_bio_ch{chapter}_sentences_BertSoftmax-1129_025337_predicted_terms.json", "r") as fid:
        terms = json.load(fid)
        stats['num_terms'].append(len(terms))
    
df = pd.DataFrame(stats)     
df['ratio'] = df['num_terms'] / df['num_sentences']
df

,chapter,num_terms,num_sentences,ratio
0,39,593,711,0.834037
1,40,838,819,1.023199
2,41,865,884,0.978507
3,42,894,934,0.957173
4,43,721,756,0.953704


In [42]:
dfs = []
for chapter in chapters:
    with open(f"{data_dir}/life_bio_ch{chapter}/life_bio_ch{chapter}_sentences.txt", "r") as fid:
        sentences = fid.readlines()
        sentences = [s.strip() for s in sentences if len(s.strip())]
        
    df = pd.DataFrame({'sentence': sentences})
    for i in range(1, 41):
        df[f"term{i}"] = '' 

    with open(f"{data_dir}/life_bio_ch{chapter}/life_bio_ch{chapter}_sentences_BertSoftmax-1129_025337_annotated_text.txt", "r") as fid:
        sentences_annotated = fid.readlines()
        sentences_annotated = [s.strip() for s in sentences_annotated if len(s.strip())]

    for ix, sa in enumerate(sentences_annotated):
        entities = re.findall('<entity>.*?</entity>', sa)
        events = re.findall('<event>.*?</event>', sa)
        events = [e.replace('<event>', '').replace('</event>', '') + ' - event' for e in set(events)]
        entities = [e.replace('<entity>', '').replace('</entity>', '') + ' - entity' for e in set(entities)]
        terms = events + entities
        for j, t in enumerate(terms):
            df.loc[ix, f"term{j}"] = t
    dfs.append(df)

In [47]:
writer = pd.ExcelWriter('../data/for_biologist_correction/terms_pre_correction.xlsx', engine='xlsxwriter')
for ch, df in zip(chapters, dfs):
    df.to_excel(writer, sheet_name=f"life_bio_ch{ch}", index=False)
writer.save()

In [48]:
df = pd.read_excel("../data/for_biologist_correction/terms_pre_correction.xlsx", sheet_name="life_bio_ch39")
df

,sentence,term1,term2,term3,term4,term5,term6,term7,term8,term9,...,term32,term33,term34,term35,term36,term37,term38,term39,term40,term0
0,"Chapter 39: Physiology, Homeostasis, and Tempe...",Temperature - entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homeostasis - event
1,"Paula Radcliffe, photographed here during her ...",heat stress - entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula Radcliffe - entity
2,"When the body is subjected to extreme heat, it...",body - entity,mechanisms - entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,heat - entity
3,Key Concepts:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Key - entity
4,Animals Are Composed of Organs Built from Four...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organs - entity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,"During periods of arousal, body temperatures (...",arousal - entity,body temperatures - entity,measured - entity,squirrels - entity,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,metabolic rates - entity
707,Recordings of two individual squirrels are sho...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,squirrels - entity
708,They are representative of the squirrels assig...,squirrels - entity,representative - entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groups - entity
709,The table reports the mean values during the r...,values - entity,metabolic rate - entity,groups - entity,body temperature - entity,error - entity,squirrels - entity,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperatures - entity
